In [11]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

Define Path 

In [12]:
# Define paths
data_dir = 'C:/License_Plate_Detection/Data/'
train_detection_dir = os.path.join(data_dir, 'license_plates_detection_train')
train_recognition_dir = os.path.join(data_dir, 'license_plates_recognition_train')
test_dir = os.path.join(data_dir, 'test')
detection_annotations_csv = os.path.join(data_dir, 'Licplatesdetection_train.csv')
recognition_annotations_csv = os.path.join(data_dir, 'Licplatesrecognition_train.csv')


In [30]:
# Load and preprocess images
def load_images_from_folder(folder, target_size=(256, 256)):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, target_size)
            images.append(img)
    return np.array(images)

def load_annotations(csv_file):
    return pd.read_csv(csv_file)

# Load datasets
train_detection_images = load_images_from_folder(os.path.join(data_path, 'license_plates_detection_train'))
train_recognition_images = load_images_from_folder(os.path.join(data_path, 'license_plates_recognition_train'))
train_detection_annotations = load_annotations(os.path.join(data_path, 'Licplatesdetection_train.csv'))
train_recognition_annotations = load_annotations(os.path.join(data_path, 'Licplatesrecognition_train.csv'))



In [13]:
# Load annotations
train_detection_annotations = pd.read_csv(detection_annotations_csv)
train_recognition_annotations = pd.read_csv(recognition_annotations_csv)

In [14]:
# Print column names to verify
print("Detection annotations columns:", train_detection_annotations.columns)
print("Recognition annotations columns:", train_recognition_annotations.columns)


Detection annotations columns: Index(['img_id', 'ymin', 'xmin', 'ymax', 'xmax'], dtype='object')
Recognition annotations columns: Index(['img_id', 'text'], dtype='object')


In [15]:
# Preprocess detection annotations
def preprocess_detection_annotations(df):
    return df[['ymin', 'xmin', 'ymax', 'xmax']].values

In [16]:
def preprocess_recognition_annotations(df, num_classes):
    labels = df['text'].values
    le = LabelEncoder()
    le.fit(labels)
    labels_encoded = le.transform(labels)
    return to_categorical(labels_encoded, num_classes=num_classes), le

# Get number of classes for recognition
num_classes = len(np.unique(train_recognition_annotations['text']))
train_recognition_annotations_preprocessed, label_encoder = preprocess_recognition_annotations(train_recognition_annotations, num_classes=num_classes)


In [17]:
# Load images and preprocess
def load_images(image_dir, annotations_df, target_size=(64, 64)):
    images = []
    labels = []
    for index, row in annotations_df.iterrows():
        img_path = os.path.join(image_dir, row['img_id'])  # Use 'img_id' as the filename
        img = Image.open(img_path).resize(target_size)
        img_array = np.array(img) / 255.0  # Normalize images
        images.append(img_array)
        labels.append(row['text'])
    return np.array(images), labels

recognition_images, recognition_labels = load_images(train_recognition_dir, train_recognition_annotations)


In [10]:
print(train_recognition_annotations.head())  # Print first few rows to inspect column names


    img_id      text
0    0.jpg  117T3989
1    1.jpg  128T8086
2   10.jpg   94T3458
3  100.jpg  133T6719
4  101.jpg   68T5979


In [18]:
# Split data
X_train, X_val, y_train, y_val = train_test_split(recognition_images, train_recognition_annotations_preprocessed, test_size=0.2, random_state=42)


In [19]:
# Create data generators
def create_data_generator(image_dir, annotations_df, target_size=(64, 64), batch_size=32):
    df = pd.DataFrame({'img_id': annotations_df['img_id'], 'text': annotations_df['text']})
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    generator = datagen.flow_from_dataframe(
        dataframe=df,
        directory=image_dir,
        x_col='img_id',
        y_col='text',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    return generator

train_generator = create_data_generator(train_recognition_dir, train_recognition_annotations)
val_generator = create_data_generator(train_recognition_dir, train_recognition_annotations)


Found 720 validated image filenames belonging to 596 classes.
Found 720 validated image filenames belonging to 596 classes.


In [20]:
# Define the recognition model
def create_recognition_model(input_shape=(64, 64, 3), num_classes=596):
    input_layer = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

recognition_model = create_recognition_model(num_classes=num_classes)


In [21]:
# Train the model
history = recognition_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=val_generator
)


Epoch 1/10


c:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 121ms/step - accuracy: 0.0022 - loss: 6.4105 - val_accuracy: 0.0111 - val_loss: 6.3810
Epoch 2/10


c:\Users\ASUS\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0111 - val_loss: 6.3810
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 99ms/step - accuracy: 0.0063 - loss: 6.3740 - val_accuracy: 0.0111 - val_loss: 6.2578
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0111 - val_loss: 6.2578
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.0000e+00 - loss: 6.2762 - val_accuracy: 0.0153 - val_loss: 6.2414
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0153 - val_loss: 6.2414
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 110ms/step - accuracy: 0.0070 - loss: 6.2330 - val_accuracy: 0.0111 - val_loss: 6.1688
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0111 - val_loss: 6.1688
Epoch 9/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - accuracy: 0.0163 - loss: 6.1832

In [22]:
# Evaluate the model
def evaluate_model(model, generator):
    results = model.evaluate(generator)
    return results

In [23]:
# Create a generator for validation data
val_generator = create_data_generator(train_recognition_dir, train_recognition_annotations)

recognition_results = evaluate_model(recognition_model, val_generator)
print('Recognition Results:', recognition_results)

Found 720 validated image filenames belonging to 596 classes.
 5/23 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.0028 - loss: 6.0880    

c:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.0067 - loss: 6.1054
Recognition Results: [6.110418796539307, 0.011111111380159855]


In [24]:
# Save results
results_dir = 'C:/License_Plate_Detection/results/'
os.makedirs(results_dir, exist_ok=True)


In [25]:
# Save model
recognition_model.save(os.path.join(results_dir, 'recognition_model.h5'))

In [26]:
# Save results to CSV
recognition_results_df = pd.DataFrame({'Metric': ['Loss', 'Accuracy'], 'Value': recognition_results})
recognition_results_df.to_csv(os.path.join(results_dir, 'recognition_results.csv'), index=False)